In [9]:
import numpy as np
import nltk
import glob
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
word_tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
 


In [10]:
with open('bramstoker.txt', 'r') as myfile:
    data=myfile.read().replace('\n', '')

In [11]:
tokens = nltk.word_tokenize(data.lower())
words = word_tokenizer.tokenize(data.lower())
sentences = sentence_tokenizer.tokenize(data)
vocab = set(words)
words_per_sentence = np.array([len(word_tokenizer.tokenize(s))
                                for s in sentences])

In [12]:
e=0
num_chapters = len(data)
fvs_lexical = np.zeros((len(data), 3), np.float64)
fvs_punct = np.zeros((len(data), 3), np.float64)

# average number of words per sentence
fvs_lexical[e, 0] = words_per_sentence.mean()
# sentence length variation
fvs_lexical[e, 1] = words_per_sentence.std()
# Lexical diversity
fvs_lexical[e, 2] = len(vocab) / float(len(words))

# Commas per sentence
fvs_punct[e, 0] = tokens.count(',') / float(len(sentences))
# Semicolons per sentence
fvs_punct[e, 1] = tokens.count(';') / float(len(sentences))
# Colons per sentence
fvs_punct[e, 2] = tokens.count(':') / float(len(sentences))

# apply whitening to decorrelate the features
fvs_lexical = whiten(fvs_lexical)
fvs_punct = whiten(fvs_punct)

In [13]:
# get most common words in the whole book
NUM_TOP_WORDS = 10
all_tokens = nltk.word_tokenize(data)
fdist = nltk.FreqDist(all_tokens)
#vocab = fdist.keys()[:NUM_TOP_WORDS]
vocab = fdist.most_common(NUM_TOP_WORDS)
 
# use sklearn to create the bag for words feature vector for each chapter
vectorizer = CountVectorizer(vocabulary=vocab, tokenizer=nltk.word_tokenize)
data = [data]
fvs_bow = vectorizer.fit_transform(data).toarray().astype(np.float64)
 
# normalise by dividing each row by its Euclidean norm
fvs_bow /= np.c_[np.apply_along_axis(np.linalg.norm, 1, fvs_bow)]

C:\Users\Jesse\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: RuntimeWarning: invalid value encountered in true_divide


In [14]:
tokens = nltk.word_tokenize(data)

TypeError: expected string or bytes-like object